In [1]:
import pandas as pd
from pandas import Series
import requests
import json

In [2]:
#import the csv file
csv_file = "csv_files/source_data/film_permits.csv"

df = pd.read_csv(csv_file, encoding = "ISO-8859-1", parse_dates=["StartDateTime", "EndDateTime"])
df.head()

,EventID,EventType,StartDateTime,EndDateTime,EnteredOn,EventAgency,ParkingHeld,Borough,CommunityBoard(s),PolicePrecinct(s),Category,SubCategoryName,Country,ZipCode(s)
0,446040,Shooting Permit,2018-10-19 14:00:00,2018-10-20 04:00:00,10/16/2018 11:57,"Mayor's Office of Film, Theatre & Broadcasting",THOMPSON STREET between PRINCE STREET and SPRI...,Manhattan,2,1,Television,Cable-episodic,United States of America,10012
1,446168,Shooting Permit,2018-10-19 14:00:00,2018-10-20 02:00:00,10/16/2018 19:03,"Mayor's Office of Film, Theatre & Broadcasting",MARBLE HILL AVENUE between WEST 227 STREET an...,Manhattan,"12, 8","34, 50",Film,Feature,United States of America,"10034, 10463"
2,186438,Shooting Permit,2014-10-30 07:00:00,2014-10-31 02:00:00,10/27/2014 12:14,"Mayor's Office of Film, Theatre & Broadcasting",LAUREL HILL BLVD between REVIEW AVENUE and RUS...,Queens,"2, 5","104, 108",Television,Episodic series,United States of America,11378
3,445255,Shooting Permit,2018-10-20 07:00:00,2018-10-20 18:00:00,10/9/2018 21:34,"Mayor's Office of Film, Theatre & Broadcasting",JORALEMON STREET between BOERUM PLACE and COUR...,Brooklyn,2,84,Still Photography,Not Applicable,United States of America,11201
4,128794,Theater Load in and Load Outs,2013-11-16 00:01:00,2013-11-17 06:00:00,11/7/2013 15:48,"Mayor's Office of Film, Theatre & Broadcasting",WEST 31 STREET between 7 AVENUE and 8 AVENUE...,Manhattan,"4, 5",14,Theater,Theater,United States of America,"10001, 10121"


In [3]:
# Add columns for start and end year
clean_df = df
clean_df["StartYear"] = pd.DatetimeIndex(clean_df["StartDateTime"]).year
clean_df["EndYear"] = pd.DatetimeIndex(clean_df["EndDateTime"]).year
clean_df.head()

,EventID,EventType,StartDateTime,EndDateTime,EnteredOn,EventAgency,ParkingHeld,Borough,CommunityBoard(s),PolicePrecinct(s),Category,SubCategoryName,Country,ZipCode(s),StartYear,EndYear
0,446040,Shooting Permit,2018-10-19 14:00:00,2018-10-20 04:00:00,10/16/2018 11:57,"Mayor's Office of Film, Theatre & Broadcasting",THOMPSON STREET between PRINCE STREET and SPRI...,Manhattan,2,1,Television,Cable-episodic,United States of America,10012,2018,2018
1,446168,Shooting Permit,2018-10-19 14:00:00,2018-10-20 02:00:00,10/16/2018 19:03,"Mayor's Office of Film, Theatre & Broadcasting",MARBLE HILL AVENUE between WEST 227 STREET an...,Manhattan,"12, 8","34, 50",Film,Feature,United States of America,"10034, 10463",2018,2018
2,186438,Shooting Permit,2014-10-30 07:00:00,2014-10-31 02:00:00,10/27/2014 12:14,"Mayor's Office of Film, Theatre & Broadcasting",LAUREL HILL BLVD between REVIEW AVENUE and RUS...,Queens,"2, 5","104, 108",Television,Episodic series,United States of America,11378,2014,2014
3,445255,Shooting Permit,2018-10-20 07:00:00,2018-10-20 18:00:00,10/9/2018 21:34,"Mayor's Office of Film, Theatre & Broadcasting",JORALEMON STREET between BOERUM PLACE and COUR...,Brooklyn,2,84,Still Photography,Not Applicable,United States of America,11201,2018,2018
4,128794,Theater Load in and Load Outs,2013-11-16 00:01:00,2013-11-17 06:00:00,11/7/2013 15:48,"Mayor's Office of Film, Theatre & Broadcasting",WEST 31 STREET between 7 AVENUE and 8 AVENUE...,Manhattan,"4, 5",14,Theater,Theater,United States of America,"10001, 10121",2013,2013


In [4]:
#create new df from selected columns
clean_df = df[["EventID", "EventType", "StartDateTime","EndDateTime", "StartYear", "EndYear", "Borough", "Category", "SubCategoryName", "Country", "ZipCode(s)"]]

clean_df.head()

,EventID,EventType,StartDateTime,EndDateTime,StartYear,EndYear,Borough,Category,SubCategoryName,Country,ZipCode(s)
0,446040,Shooting Permit,2018-10-19 14:00:00,2018-10-20 04:00:00,2018,2018,Manhattan,Television,Cable-episodic,United States of America,10012
1,446168,Shooting Permit,2018-10-19 14:00:00,2018-10-20 02:00:00,2018,2018,Manhattan,Film,Feature,United States of America,"10034, 10463"
2,186438,Shooting Permit,2014-10-30 07:00:00,2014-10-31 02:00:00,2014,2014,Queens,Television,Episodic series,United States of America,11378
3,445255,Shooting Permit,2018-10-20 07:00:00,2018-10-20 18:00:00,2018,2018,Brooklyn,Still Photography,Not Applicable,United States of America,11201
4,128794,Theater Load in and Load Outs,2013-11-16 00:01:00,2013-11-17 06:00:00,2013,2013,Manhattan,Theater,Theater,United States of America,"10001, 10121"


In [5]:
#split the zipcode on comma 
s=clean_df["ZipCode(s)"].str.split(",").apply(Series, 1).stack()

#drop index to match with clean_df index
s.index=s.index.droplevel(-1)

#name the object to make join 
s.name = "ZipCode"
s

0         10012
1         10034
1         10463
2         11378
3         11201
          ...  
67058     11363
67059     11106
67060     11101
67060     11222
67061     11201
Name: ZipCode, Length: 99644, dtype: object

In [6]:
#create another df to make the join and also check if the rows are correct
df = clean_df.join(s)
df

,EventID,EventType,StartDateTime,EndDateTime,StartYear,EndYear,Borough,Category,SubCategoryName,Country,ZipCode(s),ZipCode
0,446040,Shooting Permit,2018-10-19 14:00:00,2018-10-20 04:00:00,2018,2018,Manhattan,Television,Cable-episodic,United States of America,10012,10012
1,446168,Shooting Permit,2018-10-19 14:00:00,2018-10-20 02:00:00,2018,2018,Manhattan,Film,Feature,United States of America,"10034, 10463",10034
1,446168,Shooting Permit,2018-10-19 14:00:00,2018-10-20 02:00:00,2018,2018,Manhattan,Film,Feature,United States of America,"10034, 10463",10463
2,186438,Shooting Permit,2014-10-30 07:00:00,2014-10-31 02:00:00,2014,2014,Queens,Television,Episodic series,United States of America,11378,11378
3,445255,Shooting Permit,2018-10-20 07:00:00,2018-10-20 18:00:00,2018,2018,Brooklyn,Still Photography,Not Applicable,United States of America,11201,11201
...,...,...,...,...,...,...,...,...,...,...,...,...
67058,461508,Shooting Permit,2019-01-25 07:00:00,2019-01-25 22:00:00,2019,2019,Queens,Television,Episodic series,United States of America,"11222, 11363",11363
67059,545945,Shooting Permit,2020-09-10 07:00:00,2020-09-10 22:00:00,2020,2020,Queens,Television,Cable-episodic,United States of America,11106,11106
67060,466494,Shooting Permit,2019-02-28 07:00:00,2019-02-28 21:00:00,2019,2019,Queens,Television,Episodic series,United States of America,"11101, 11222",11101
67060,466494,Shooting Permit,2019-02-28 07:00:00,2019-02-28 21:00:00,2019,2019,Queens,Television,Episodic series,United States of America,"11101, 11222",11222


In [7]:
# View variable types
df.dtypes

EventID                     int64
EventType                  object
StartDateTime      datetime64[ns]
EndDateTime        datetime64[ns]
StartYear                   int64
EndYear                     int64
Borough                    object
Category                   object
SubCategoryName            object
Country                    object
ZipCode(s)                 object
ZipCode                    object
dtype: object

In [8]:
#drop the ZipCode(s) column
del df["ZipCode(s)"]

In [9]:
#check
df.head()

,EventID,EventType,StartDateTime,EndDateTime,StartYear,EndYear,Borough,Category,SubCategoryName,Country,ZipCode
0,446040,Shooting Permit,2018-10-19 14:00:00,2018-10-20 04:00:00,2018,2018,Manhattan,Television,Cable-episodic,United States of America,10012
1,446168,Shooting Permit,2018-10-19 14:00:00,2018-10-20 02:00:00,2018,2018,Manhattan,Film,Feature,United States of America,10034
1,446168,Shooting Permit,2018-10-19 14:00:00,2018-10-20 02:00:00,2018,2018,Manhattan,Film,Feature,United States of America,10463
2,186438,Shooting Permit,2014-10-30 07:00:00,2014-10-31 02:00:00,2014,2014,Queens,Television,Episodic series,United States of America,11378
3,445255,Shooting Permit,2018-10-20 07:00:00,2018-10-20 18:00:00,2018,2018,Brooklyn,Still Photography,Not Applicable,United States of America,11201


In [10]:
# Make API call to OpenDataSoft public API to pull latitude and longitude for all New York zip codes

url = "https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=&rows=2500&facet=state&facet=timezone&facet=dst&refine.state=NY"
zip_data = requests.get(url).json()
zip_data

{'nhits': 2281,
 'parameters': {'dataset': 'us-zip-code-latitude-and-longitude',
  'timezone': 'UTC',
  'rows': 2500,
  'start': 0,
  'format': 'json',
  'facet': ['state', 'timezone', 'dst']},
 'records': [{'datasetid': 'us-zip-code-latitude-and-longitude',
   'recordid': '6e589b15cb714258c5710936f33ed6100774d2c2',
   'fields': {'city': 'Henderson',
    'zip': '13650',
    'dst': 1,
    'geopoint': [43.82212, -76.2127],
    'longitude': -76.2127,
    'state': 'NY',
    'latitude': 43.82212,
    'timezone': -5},
   'geometry': {'type': 'Point', 'coordinates': [-76.2127, 43.82212]},
   'record_timestamp': '2018-02-09T16:33:38.603000+00:00'},
  {'datasetid': 'us-zip-code-latitude-and-longitude',
   'recordid': '3e20dbd8e01f4f86f644bbbd3598fda2a3af163a',
   'fields': {'city': 'Buffalo',
    'zip': '14219',
    'dst': 1,
    'geopoint': [42.790708, -78.82717],
    'longitude': -78.82717,
    'state': 'NY',
    'latitude': 42.790708,
    'timezone': -5},
   'geometry': {'type': 'Point', 'co

In [11]:
# Loop through all the zip codes in New York
# Add the zip, latitude, and longitude to a dictionary
# Append the dictionary to a list

zip_dict_list = []
i = 0

for record in zip_data["records"]:
    zip_code = zip_data["records"][i]["fields"]["zip"]
    latitude = zip_data["records"][i]["fields"]["latitude"]
    longitude = zip_data["records"][i]["fields"]["longitude"]
    #geopoint = zip_data["records"][i]["fields"]["geopoint"]
    zip_dict = {"Zip": zip_code,
                "Latitude": latitude,
                "Longitude": longitude
                #,"Geopoint": geopoint
               }
    zip_dict_list.append(zip_dict)
    i += 1

In [19]:
# Create a dataframe of New York zip codes
zip_df = pd.DataFrame(zip_dict_list)
zip_df.head()

,Zip,Latitude,Longitude
0,13650,43.822120,-76.212700
1,14219,42.790708,-78.827170
2,14872,42.218224,-76.846940
3,10286,40.714231,-74.011895
4,12230,42.614852,-73.970812


In [13]:
# Strip the zip code string to remove extra whitespace
df['ZipCode'] = df['ZipCode'].astype(str)
df['ZipCode'] = df['ZipCode'].str.strip()

In [14]:
# Strip the zip code string to remove extra whitespace
zip_df['Zip'] = zip_df['Zip'].astype(str)
zip_df['Zip'] = zip_df['Zip'].str.strip()

In [18]:
# Perform an inner join to merge the data
merge_df = pd.merge(df, zip_df, left_on="ZipCode", right_on="Zip", how="left")
merge_df.head()

,EventID,EventType,StartDateTime,EndDateTime,StartYear,EndYear,Borough,Category,SubCategoryName,Country,ZipCode,Zip,Latitude,Longitude
0,446040,Shooting Permit,2018-10-19 14:00:00,2018-10-20 04:00:00,2018,2018,Manhattan,Television,Cable-episodic,United States of America,10012,10012,40.725960,-73.99834
1,446168,Shooting Permit,2018-10-19 14:00:00,2018-10-20 02:00:00,2018,2018,Manhattan,Film,Feature,United States of America,10034,10034,40.867653,-73.92000
2,446168,Shooting Permit,2018-10-19 14:00:00,2018-10-20 02:00:00,2018,2018,Manhattan,Film,Feature,United States of America,10463,10463,40.881086,-73.90749
3,186438,Shooting Permit,2014-10-30 07:00:00,2014-10-31 02:00:00,2014,2014,Queens,Television,Episodic series,United States of America,11378,11378,40.724096,-73.90271
4,445255,Shooting Permit,2018-10-20 07:00:00,2018-10-20 18:00:00,2018,2018,Brooklyn,Still Photography,Not Applicable,United States of America,11201,11201,40.695286,-73.98907


In [16]:
# Drop duplicate zip code column
merge_df = merge_df.drop(columns="Zip")
merge_df.head()

,EventID,EventType,StartDateTime,EndDateTime,StartYear,EndYear,Borough,Category,SubCategoryName,Country,ZipCode,Latitude,Longitude
0,446040,Shooting Permit,2018-10-19 14:00:00,2018-10-20 04:00:00,2018,2018,Manhattan,Television,Cable-episodic,United States of America,10012,40.725960,-73.99834
1,446168,Shooting Permit,2018-10-19 14:00:00,2018-10-20 02:00:00,2018,2018,Manhattan,Film,Feature,United States of America,10034,40.867653,-73.92000
2,446168,Shooting Permit,2018-10-19 14:00:00,2018-10-20 02:00:00,2018,2018,Manhattan,Film,Feature,United States of America,10463,40.881086,-73.90749
3,186438,Shooting Permit,2014-10-30 07:00:00,2014-10-31 02:00:00,2014,2014,Queens,Television,Episodic series,United States of America,11378,40.724096,-73.90271
4,445255,Shooting Permit,2018-10-20 07:00:00,2018-10-20 18:00:00,2018,2018,Brooklyn,Still Photography,Not Applicable,United States of America,11201,40.695286,-73.98907


In [17]:
# Export the cleaned data to a csv
merge_df.to_csv("csv_files/cleaned_data/film_permits_clean.csv")

In [20]:
# Group by category and count number of unique event IDs for each category
grouped_df = merge_df.groupby(["Category"])
event_df = grouped_df["EventID"].nunique()
event_df

Category
Commercial              5278
Documentary              282
Film                   10855
Music Video              221
Red Carpet/Premiere        1
Still Photography       4011
Student                  426
Television             36802
Theater                 6749
WEB                     2437
Name: EventID, dtype: int64

In [23]:
# Export to csv
event_df.to_csv("csv_files/cleaned_data/permits_by_category.csv")

In [22]:
# Group by category and subcategory and count number of unique event IDs for each subcategory
grouped_df2 = merge_df.groupby(["Category", "SubCategoryName"])
event_df2 = grouped_df2["EventID"].nunique()
event_df2

Category             SubCategoryName        
Commercial           Commercial                  4232
                     Industrial/Corporate         311
                     Not Applicable                 2
                     PSA                          129
                     Promo                        599
                     Short                          2
                     Trailers                       3
Documentary          Not Applicable               282
Film                 Feature                     9095
                     Not Applicable              1036
                     Short                        724
Music Video          Independent Artist            37
                     Not Applicable                25
                     Signed Artist                159
Red Carpet/Premiere  TV Premiere                    1
Still Photography    Commercial                     2
                     Not Applicable              4009
Student              Not Applicable  

In [24]:
# Export to csv
event_df.to_csv("csv_files/cleaned_data/permits_by_category_and_subcategory.csv")

In [25]:
# Group by category and subcategory and count number of unique event IDs for each subcategory
grouped_df3 = merge_df.groupby(["StartYear", "Category"])
event_df3 = grouped_df3["EventID"].nunique()
event_df3

StartYear  Category         
2012       Commercial            717
           Documentary            35
           Film                 1445
           Music Video            27
           Still Photography     458
                                ... 
2020       Still Photography      26
           Student                18
           Television            949
           Theater               170
           WEB                    43
Name: EventID, Length: 82, dtype: int64

In [26]:
# Export the cleaned data to a csv
event_df3.to_csv("csv_files/cleaned_data/permits_by_year_and_category.csv")